In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
import json

f = open('data/kobis.json')
data = json.load(f)

for i in range(2022, 1970, -1):
    ranking_year = str(i)
    print("연도: " + ranking_year)
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        title = movie['title']
        year = movie['year']
        # date = movie['date']
        print(f"순위: {ranking}, 영화 제목: {title}, 개봉 연도: {year}")
    print("\n\n")
f.close()

연도: 2022
순위: 1, 영화 제목: 범죄도시 2, 개봉 연도: 2022
순위: 2, 영화 제목: 탑건: 매버릭, 개봉 연도: 2022
순위: 3, 영화 제목: 한산: 용의 출현, 개봉 연도: 2022
순위: 4, 영화 제목: 공조2: 인터내셔날, 개봉 연도: 2022
순위: 5, 영화 제목: 닥터 스트레인지: 대혼돈의 멀티버스, 개봉 연도: 2022
순위: 6, 영화 제목: 헌트, 개봉 연도: 2022
순위: 7, 영화 제목: 쥬라기 월드: 도미니언, 개봉 연도: 2022
순위: 8, 영화 제목: 마녀(魔女) Part2. The Other One, 개봉 연도: 2022
순위: 9, 영화 제목: 토르: 러브 앤 썬더, 개봉 연도: 2022
순위: 10, 영화 제목: 미니언즈2, 개봉 연도: 2022
순위: 11, 영화 제목: 비상선언, 개봉 연도: 2022
순위: 12, 영화 제목: 스파이더맨: 노 웨이 홈, 개봉 연도: 2021
순위: 13, 영화 제목: 육사오(6/45), 개봉 연도: 2022
순위: 14, 영화 제목: 헤어질 결심, 개봉 연도: 2022
순위: 15, 영화 제목: 외계+인 1부, 개봉 연도: 2022
순위: 16, 영화 제목: 해적: 도깨비 깃발, 개봉 연도: 2022
순위: 17, 영화 제목: 브로커, 개봉 연도: 2022
순위: 18, 영화 제목: 신비한 동물들과 덤블도어의 비밀, 개봉 연도: 2022
순위: 19, 영화 제목: 인생은 아름다워, 개봉 연도: 2022
순위: 20, 영화 제목: 더 배트맨, 개봉 연도: 2022
순위: 21, 영화 제목: 정직한 후보2, 개봉 연도: 2022
순위: 22, 영화 제목: 씽2게더, 개봉 연도: 2022
순위: 23, 영화 제목: 킹메이커, 개봉 연도: 2022
순위: 24, 영화 제목: 언차티드, 개봉 연도: 2022
순위: 25, 영화 제목: 극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교, 개봉 연도: 2022
순위: 26, 영화 제목: 경관의 피, 개봉 연도: 2022
순위:

In [3]:
chrome_path = 'chromedriver'

In [4]:
from selenium.webdriver.common.by import By

In [6]:
import numpy as np

In [62]:
def search_movie(driver, query, release_year):
    #영화 제목 검색 
    driver.find_element(By.XPATH , '//*[@id="ipt_tx_srch"]').click()
    element = driver.find_element("id","ipt_tx_srch")
    element.send_keys(str(query))
    element.send_keys("\n")

    href = driver.find_element(By.XPATH, '//*[@class="search_menu"]/li[2]/a').get_attribute('href')
    driver.get(href)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    try:
        movie_list = soup.find('div', id = 'old_content').find('ul', class_="search_list_1").find_all('li')
    except:
        print("no search result")
        return None
        # 기존 query를 약간 짧게 만들어서 다시 검색
        # soup 가져와서
        #        movie_list = soup.find('div', id = 'old_content').find('ul', class_="search_list_1").find_all('li')
        # return


    if (len(movie_list) == 1):
        href = movie_list[0].find('a')['href']
        movie_url = 'https://movie.naver.com' + href
        return movie_url

    year_list = []
    for movie_li in movie_list:
        title = movie_li.find('dt').text

        paren = title.rfind('(')

        if paren != -1:
            title = title[:paren-1]
        
        title_cleaned = title.replace(' ', '').replace('-', '').replace(':', '').replace('.', '').replace(',','').replace('\'', '')
        query_cleaned = str(query).replace(' ', '').replace('-', '').replace(':', '').replace('.', '').replace(',','').replace('\'', '').replace(str(year), '')

        if (title_cleaned.startswith(query_cleaned)):
            made_year = movie_li.find('dd', class_ = 'etc').find_all('a')[-1].text
            if made_year == '':
                year_list.append(2000)
            elif (release_year - int(made_year) < 0):
                year_list.append(2000)
            else:
                year_list.append(release_year - int(made_year))
        else:
            year_list.append(2000)
            continue

    index_min = np.argmin(year_list)

    if (year_list[index_min] == 2000):
        print("no match for search result")
        return None

    href = movie_list[index_min].find('a')['href']
    movie_url = 'https://movie.naver.com' + href

    return movie_url


In [ ]:
f = open('data/kobis.json')
data = json.load(f)
f.close()
url_data = {}

driver = webdriver.Chrome(chrome_path)
url = "https://movie.naver.com"
driver.get(url)

for i in range(2022, 1970, -1):
    ranking_year = str(i)
    movies = {}
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        title = str(movie['title'])
        year = movie['year']
        if (year == np.nan):
            movies[ranking] = {"title": title, "year": year, "url": None}
            continue
            
        url = search_movie(driver, title, year)
        if url == None:
            print(title, year)
        movies[ranking] = {"title": title, "year": year, "url": url}
    url_data[ranking_year] = movies
    print(f"year {year} added")

driver.quit()

with open("data/urls.json", 'w', encoding="UTF-8") as file:
    json.dump(url_data, file, indent=4, ensure_ascii=False)

In [28]:
with open("data/urls.json", 'w', encoding="UTF-8") as file:
    json.dump(url_data, file, indent=4, ensure_ascii=False)

In [29]:
f = open('data/kobis.json')
data = json.load(f)
f.close()
url_data = {}

driver = webdriver.Chrome(chrome_path)
url = "https://movie.naver.com"
driver.get(url)

for i in range(2010, 2009, -1):
    ranking_year = str(i)
    movies = {}
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        title = str(movie['title'])
        year = movie['year']
        print(str(title), str(year))
        if (year == np.nan):
            movies[ranking] = {"title": title, "year": year, "url": None}
            print("year is nan")
            continue
            
        url = search_movie(driver, title, year)
        if url == None:
            print(title, year)
        movies[ranking] = {"title": title, "year": year, "url": url}
    url_data[ranking_year] = movies
    print(f"year {year} added")

driver.quit()

아바타 2009
아저씨 2010
인셉션 2010
의형제 2010
아이언맨 2 2010
전우치 2009
이끼 2010
포화속으로 2010
하모니 2010
방자전 2010
솔트 2010
부당거래 2010
시라노; 연애조작단 2010
타이탄 2010
드래곤 길들이기 2010
해리 포터와 죽음의 성물1 2010
no match for search result
해리 포터와 죽음의 성물1 2010
하녀 2010
나잇&데이 2010
슈렉 포에버 2010
이상한 나라의 앨리스 2010
초능력자 2010
이클립스 2010
쩨쩨한 로맨스 2010
페르시아의 왕자: 시간의 모래 2010
퍼시 잭슨과 번개 도둑 2010
해결사 2010
악마를 보았다 2010
로빈 후드 2010
무적자 2010
토이 스토리 3 2010
구르믈 버서난 달처럼 2010
황해 2010
라스트 에어벤더 2010
헬로우 고스트 2010
육혈포 강도단 2010
심야의 F.M. 2010
no match for search result
심야의 F.M. 2010
레지던트 이블 4: 끝나지 않은 전쟁 3D 2010
나니아 연대기: 새벽 출정호의 항해 2010
용서는 없다 2010
베스트셀러 2010
김종욱 찾기 2010
마루 밑 아리에티 2010
셜록 홈즈 2009
슈퍼 배드 2010
파괴된 사나이 2010
셔터 아일랜드 2010
방가? 방가! 2010
평행이론 2010
투어리스트 2010
스카이 라인 2010
고死 두 번째 이야기: 교생실습 2010
no match for search result
고死 두 번째 이야기: 교생실습 2010
A-특공대 2010
주유소 습격사건 2 2010
마음이2 2010
내 깡패 같은 애인 2010
스텝 업 3D 2010
하늘에서 음식이 내린다면 2010
오션스 2010
명탐정 코난: 천공의 난파선 2010
새미의 어드벤쳐 2010
그린 존 2010
이층의 악당 2010
반가운 살인자 2010
레터스 투 줄리엣 2010
퀴즈왕 2010
마법사의 제자 2010
앨빈과 슈퍼밴드2 200

In [30]:
f = open('data/urls.json')
data = json.load(f)
f.close()

data["2010"] = url_data["2010"]

with open("data/urls.json", 'w', encoding="UTF-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [38]:
f = open('data/kobis.json')
data = json.load(f)
f.close()
url_data = {}

driver = webdriver.Chrome(chrome_path)
url = "https://movie.naver.com"
driver.get(url)

for i in range(2009, 1970, -1):
    ranking_year = str(i)
    movies = {}
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        title = str(movie['title'])
        year = movie['year']
        # print(str(title), str(year))
        if (year == np.nan):
            movies[ranking] = {"title": title, "year": year, "url": None}
            print("year is nan")
            continue
            
        url = search_movie(driver, title, year)
        if url == None:
            print(title, year)
        movies[ranking] = {"title": title, "year": year, "url": url}
    url_data[ranking_year] = movies
    print(f"year {year} added")

driver.quit()

no match for search result
마더 2009
no match for search result
시크릿 2009
no match for search result
실종 2009
year 2009 added
no match for search result
P.S. 아이 러브 유 2008
year 2008 added
no match for search result
리턴 2007
no search result
부그와 엘리엇(애니) 2007
year 2007 added
no match for search result
아파트 2006
no match for search result
파이널 데스티네이션 2006
year 2006 added
no search result
로봇(애니) 2005
no match for search result
외출 2005
no match for search result
숨바꼭질 2005
no search result
트리플 X2 : 넥스트 레벨 2005
no search result
유령신부(애니) 2005
year 2005 added
no match for search result
가족 2004
no match for search result
페이스 2004
no match for search result
에이리언 VS 프레데터 2004
no match for search result
쓰리 몬스터 2004
year 2004 added
no match for search result
주온 2 2003
no match for search result
더 링 2003
no match for search result
더 코어 2003
no match for search result
블루 2003
year 2003 added
no match for search result
챔피언 2002
no match for search result
결혼은, 미친짓이다 2002
no match for search result
해적 디스코왕 되다 20

In [41]:
f = open('data/urls.json')
data = json.load(f)
f.close()

for year in url_data:
    data[year] = url_data[year]

with open("data/urls.json", 'w', encoding="UTF-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [44]:
f = open('data/urls.json')
data = json.load(f)
f.close()

for i in range(2022, 1970, -1):
    ranking_year = str(i)
    count = 0
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        url = movie['url']
        # print(str(title), str(year))

        if url == None:
            count += 1
    print(f"year {ranking_year}: {count}")
            

year 2022: 1
year 2021: 0
year 2020: 1
year 2019: 0
year 2018: 0
year 2017: 0
year 2016: 0
year 2015: 1
year 2014: 3
year 2013: 1
year 2012: 3
year 2011: 3
year 2010: 5
year 2009: 3
year 2008: 1
year 2007: 2
year 2006: 2
year 2005: 5
year 2004: 4
year 2003: 4
year 2002: 8
year 2001: 7
year 2000: 6
year 1999: 4
year 1998: 6
year 1997: 10
year 1996: 7
year 1995: 2
year 1994: 3
year 1993: 9
year 1992: 10
year 1991: 12
year 1990: 2
year 1989: 8
year 1988: 8
year 1987: 10
year 1986: 3
year 1985: 3
year 1984: 6
year 1983: 6
year 1982: 6
year 1981: 6
year 1980: 11
year 1979: 5
year 1978: 7
year 1977: 5
year 1976: 17
year 1975: 17
year 1974: 16
year 1973: 20
year 1972: 29
year 1971: 30


In [65]:
f = open('data/urls.json')
data = json.load(f)
f.close()

driver = webdriver.Chrome(chrome_path)
url = "https://movie.naver.com"
driver.get(url)

for i in range(2022, 1970, -1):
    ranking_year = str(i)
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        url = movie['url']

        if url == None:
            title = str(movie['title'])
            year = movie['year']
            print(str(title), str(year))

            if (len(title) <= 2):
                url = search_movie(driver, title+str(year), year)
            else:
                url = search_movie(driver, str(title).replace('(애니)', '').replace('\'', ''), year)
            if url == None:
                print(title, year)
            data[ranking_year][ranking] = {"title": title, "year": year, "url": url}
            continue

driver.quit()

소리꾼 2020
no match for search result
소리꾼 2020
몬스터 2014
no match for search result
몬스터 2014
헨젤과 그레텔: 마녀 사냥꾼 3D 2013
no search result
헨젤과 그레텔: 마녀 사냥꾼 3D 2013
시크릿 2009
no match for search result
시크릿 2009
아파트 2006
no match for search result
아파트 2006
파이널 데스티네이션 2006
no match for search result
파이널 데스티네이션 2006
숨바꼭질 2005
no match for search result
숨바꼭질 2005
트리플 X2 : 넥스트 레벨 2005
no search result
트리플 X2 : 넥스트 레벨 2005
페이스 2004
no match for search result
페이스 2004
주온 2 2003
no match for search result
주온 2 2003
더 링 2003
no match for search result
더 링 2003
더 코어 2003
no match for search result
더 코어 2003
블루 2003
no search result
블루 2003
챔피언 2002
no match for search result
챔피언 2002
비밀 2002
no search result
비밀 2002
패밀리 2002
no match for search result
패밀리 2002
릴로와 스티치 2002
no match for search result
릴로와 스티치 2002
오원 장승업 취화선 2002
no search result
오원 장승업 취화선 2002
A.I. 2001
no match for search result
A.I. 2001
하루 2001
no search result
하루 2001
더 기프트 2001
no match for search result
더 기프트 2001
분노의 질주 2001
no matc

In [66]:
with open("data/urls.json", 'w', encoding="UTF-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [67]:
f = open('data/urls.json')
data = json.load(f)
f.close()

url_data = {}

sum = 0
for i in range(2022, 1970, -1):
    ranking_year = str(i)
    count = 0
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        url = movie['url']
        # print(str(title), str(year))

        if url == None:
            count += 1
    sum += count
    print(f"year {ranking_year}: {count} movies without url")
print(f"total: {sum}")

year 2022: 0 movies without url
year 2021: 0 movies without url
year 2020: 1 movies without url
year 2019: 0 movies without url
year 2018: 0 movies without url
year 2017: 0 movies without url
year 2016: 0 movies without url
year 2015: 0 movies without url
year 2014: 1 movies without url
year 2013: 1 movies without url
year 2012: 0 movies without url
year 2011: 0 movies without url
year 2010: 0 movies without url
year 2009: 1 movies without url
year 2008: 0 movies without url
year 2007: 0 movies without url
year 2006: 2 movies without url
year 2005: 2 movies without url
year 2004: 1 movies without url
year 2003: 4 movies without url
year 2002: 5 movies without url
year 2001: 5 movies without url
year 2000: 6 movies without url
year 1999: 3 movies without url
year 1998: 4 movies without url
year 1997: 9 movies without url
year 1996: 7 movies without url
year 1995: 1 movies without url
year 1994: 3 movies without url
year 1993: 8 movies without url
year 1992: 10 movies without url
year 19

In [68]:
f = open('data/kobis(audience).json')
data = json.load(f)
f.close()


f = open('data/urls.json')
url_data = json.load(f)
f.close()

sum = 0
for i in range(2022, 1970, -1):
    ranking_year = str(i)
    count = 0
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        movie['url'] = url_data[ranking_year][ranking]['url']


In [70]:
with open("data/urls.json", 'w', encoding="UTF-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [71]:
f = open('data/urls.json')
data = json.load(f)
f.close()

url_data = {}

sum = 0
for i in range(2022, 1979, -1):
    ranking_year = str(i)
    count = 0
    for j in range(1, 101, 1):
        ranking = str(j)
        movie = data[ranking_year][ranking]
        url = movie['url']
        # print(str(title), str(year))

        if url == None:
            count += 1
    sum += count
    print(f"year {ranking_year}: {count} movies without url")
print(f"total: {sum}")

year 2022: 0 movies without url
year 2021: 0 movies without url
year 2020: 1 movies without url
year 2019: 0 movies without url
year 2018: 0 movies without url
year 2017: 0 movies without url
year 2016: 0 movies without url
year 2015: 0 movies without url
year 2014: 1 movies without url
year 2013: 1 movies without url
year 2012: 0 movies without url
year 2011: 0 movies without url
year 2010: 0 movies without url
year 2009: 1 movies without url
year 2008: 0 movies without url
year 2007: 0 movies without url
year 2006: 2 movies without url
year 2005: 2 movies without url
year 2004: 1 movies without url
year 2003: 4 movies without url
year 2002: 5 movies without url
year 2001: 5 movies without url
year 2000: 6 movies without url
year 1999: 3 movies without url
year 1998: 4 movies without url
year 1997: 9 movies without url
year 1996: 7 movies without url
year 1995: 1 movies without url
year 1994: 3 movies without url
year 1993: 8 movies without url
year 1992: 10 movies without url
year 19